# exchange

> This module interacts with exchange

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp junk_checker

In [ ]:
#| export
import imaplib
import email
from email.header import decode_header
from datetime import datetime
import os

Run the following in exchange to export your exchange user name and password (restart Terminal and Jupyter lab if it does not work):

```
echo 'export EXCHANGE_USER="YOUR_USERNAME"' >> ~/.bashrc && grep -q 'source ~/.bashrc' ~/.bash_profile || echo -e "\nif [ -f ~/.bashrc ]; then\n  source ~/.bashrc\nfi" >> ~/.bash_profile && source ~/.bashrc
```
and

```
echo 'export EXCHANGE_PASSWORD="YOUR_PASSWORD"' >> ~/.bashrc && grep -q 'source ~/.bashrc' ~/.bash_profile || echo -e "\nif [ -f ~/.bashrc ]; then\n  source ~/.bashrc\nfi" >> ~/.bash_profile && source ~/.bashrc
```

In [ ]:
#| export
class EmailObject:
    def __init__(self, sender, subject, email_id):
        self.sender = sender
        self.subject = subject
        self.email_id = email_id

class EmailClient:
    def __init__(self, server, port, username, password):
        self.mail = imaplib.IMAP4_SSL(server, port)
        self.mail.login(username, password)

    def get_emails(self, since_date=None):
        self.mail.select("inbox")
        
        if since_date:
            since_date_str = since_date.strftime('%d-%b-%Y')
            status, messages = self.mail.search(None, f'SINCE {since_date_str}')
        else:
            status, messages = self.mail.search(None, "ALL")
            
        email_ids = messages[0].split()
        email_objects = []
        
        for e_id in email_ids:
            status, msg_data = self.mail.fetch(e_id, "(RFC822)")
            msg = email.message_from_bytes(msg_data[0][1])
            
            # Decode subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                subject = subject.decode(encoding if encoding else "utf-8")
            
            # Extract sender
            sender = msg["From"]
            
            # Create email object and add to list
            email_objects.append(EmailObject(sender, subject, e_id))
        
        return email_objects

    def move_email(self, email_object, target_folder):
        email_id = email_object.email_id
        
        # Ensure email_id is a byte-string
        if not isinstance(email_id, bytes):
            email_id = email_id.encode('utf-8')
            
        result = self.mail.copy(email_id, target_folder)
        if result[0] == 'OK':
            self.mail.store(email_id, '+FLAGS', '(\Deleted)')
            self.mail.expunge()

    def logout(self):
        self.mail.logout()

In [ ]:
#| notest

USERNAME = os.environ.get("EXCHANGE_USER")
PASSWORD = os.environ.get("EXCHANGE_PASSWORD")

# Usage example
email_client = EmailClient("msx.tu-dresden.de", 993, USERNAME, PASSWORD)

# Fetch emails since a specific date
since_date = datetime(2023, 9, 22)  # Replace with actual date
emails = email_client.get_emails(since_date=since_date)

# Print and move emails
for e in emails:
    print(f"Subject: {e.subject}, ID: {e.email_id}")
    email_client.move_email(e, 'Review')

# Logout to close the connection
email_client.logout()

Subject: [TRR265] , ID: b'12954'
Subject: AW: SSRT etc, ID: b'12955'
Subject: Hilmar, we found new jobs you may be interested in, ID: b'12956'
Subject: monday - window cleaning, ID: b'12957'
Subject: RE: SSRT etc, ID: b'12958'
Subject: Individuelle Prognose für Ihren Gewichtsverlust, ID: b'12959'
